In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Main tools we used:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Models:
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

# Python utilities:
import time
import os
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

#XGBoost
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#读取数据集
df = pd.read_csv('../input/GiveMeSomeCredit/cs-training.csv')
test_df = pd.read_csv('../input/GiveMeSomeCredit/cs-test.csv')
list(df)

 **数据集属性有：**

Unnamed: 0

SeriousDlqin2yrs

RevolvingUtilizationOfUnsecuredLines
age

NumberOfTime30-59DaysPastDueNotWorse

DebtRatio

MonthlyIncome

NumberOfOpenCreditLinesAndLoans

NumberOfTimes90DaysLate

NumberRealEstateLoansOrLines

NumberOfTime60-89DaysPastDueNotWorse

NumberOfDependents

In [ ]:
#查看前五行数据
df.head()

# 1.对数据集做一些基本判断与处理

**（1）年龄是一项值得考虑的因素**

In [ ]:
#绘制 年龄/违约与否 直方图
age_hist = df['age'].hist(by=df['SeriousDlqin2yrs'], bins=20, layout=(2,1))
age_hist[0].set_xlim((0,100))
age_hist[0].set_title('SeriousDlqin2yrs = 0')
age_hist[1].set_xlim((0,100))
age_hist[1].set_title('SeriousDlqin2yrs = 1')

显然年轻人更容易违约，因此我们暂时不需要对老年人贷款施加诸多限制。

**（2）接下来，我们观察有没有异常的债务比例**

In [ ]:
df.DebtRatio.describe()

竟然有人的债务是自己资产的33万倍以上。当我们查看一些债务比例过高的人时可以得到：

In [ ]:
df.DebtRatio.quantile([.95])

In [ ]:
df[df['DebtRatio'] > 2449.0][['SeriousDlqin2yrs','MonthlyIncome']].describe()

在4000个借款比率大于3500的记录中，只有185个记录的月收入值。这些收入都在0和1之间，可以全部理解为录入错误。

0.95是我们猜测的值。为了提高准确率，我们多尝试几个值。

In [ ]:
df.DebtRatio.quantile([.86])

In [ ]:
df[df['DebtRatio'] >  453.0][['SeriousDlqin2yrs','MonthlyIncome']].describe()

In [ ]:
df.DebtRatio.quantile([.85])

In [ ]:
df[df['DebtRatio'] >  269.15][['SeriousDlqin2yrs','MonthlyIncome']].describe()

最终，我们要删除掉'DebtRatio' > 453.0的数据

**（3）现在检查 NumberOfTimes90DaysLate。**

In [ ]:
df.groupby('NumberOfTimes90DaysLate').NumberOfTimes90DaysLate.count()

In [ ]:
df[df['NumberOfTimes90DaysLate'] > 95][['SeriousDlqin2yrs','NumberOfTime60-89DaysPastDueNotWorse','NumberOfTime30-59DaysPastDueNotWorse','NumberOfTimes90DaysLate']].describe()

有264人都违约98次，这个数据显然是不正常的
考虑到确实有人违规次数很多，我们将违约96次和98次的都视为违约20次，一次保证数据不至于这么异常

（4）查看欠款和信用额度的比例

In [ ]:
df[(df['RevolvingUtilizationOfUnsecuredLines'] > .9) & (df['RevolvingUtilizationOfUnsecuredLines'] <= 4)].SeriousDlqin2yrs.describe()

In [ ]:
df[df['RevolvingUtilizationOfUnsecuredLines'] > 10].describe()

这些人欠款/信用额度的比例达到10以上，但这241人的SeriousDlqin2yrs数据看起来没有什么异常，可能是RevolvingUtilizationOfUnsecuredLines值错误，应当删除。

**（5）检查缺失值**

In [ ]:
df.isnull().sum()

应当处理MonthlyIncome和NumberOfDependents的缺失值

In [ ]:
not_missing = df.dropna()
target = 'MonthlyIncome'
predictors = [c for c in list(not_missing) if c not in [target, 'Unnamed: 0','SeriousDlqin2yrs']]
X_data = not_missing[predictors]
y_data = not_missing[target]
regr = LinearRegression().fit(X_data, y_data)

In [ ]:
regr.score(X_data, y_data)

由于此项极值很大，最好用中值填补MonthlyIncome。
用0填补NumberOfDependents。

# 二、根据以上分析，填补或调整数据集中的值

In [ ]:
# 根据（2）删除DebtRatio > 453.0的数据
removed_debt_outliers = df.drop(df[df['DebtRatio'] > 453.0].index)
removed_debt_outliers = removed_debt_outliers.fillna(removed_debt_outliers.median())

In [ ]:
#根据（4）删除'RevolvingUtilizationOfUnsecuredLines' > 10的数据
dfus = removed_debt_outliers.drop(removed_debt_outliers[removed_debt_outliers['RevolvingUtilizationOfUnsecuredLines'] > 10].index)

In [ ]:
# 根据（3）调整NumberOfTimes90DaysLate
dfn98 = dfus.copy()
dfn98.loc[dfn98['NumberOfTime30-59DaysPastDueNotWorse'] > 90, 'NumberOfTime30-59DaysPastDueNotWorse'] = 18
dfn98.loc[dfn98['NumberOfTime60-89DaysPastDueNotWorse'] > 90, 'NumberOfTime60-89DaysPastDueNotWorse'] = 18
dfn98.loc[dfn98['NumberOfTimes90DaysLate'] > 90, 'NumberOfTimes90DaysLate'] = 18

In [ ]:
#根据（5） 填补MonthlyIncome
income_median = df['MonthlyIncome'].median()
income_null_count = df['MonthlyIncome'].isnull().sum()
income_null_list = np.array([income_median]*income_null_count)
df['MonthlyIncome'][np.isnan(df['MonthlyIncome'])] = income_null_list
df['MonthlyIncome'] = df['MonthlyIncome'].astype(int)

In [ ]:
#根据（5） 填补NumberOfDependents
dependents_null_count = df['NumberOfDependents'].isnull().sum()
dependents_null_list = np.array([0]*dependents_null_count)
df['NumberOfDependents'][np.isnan(df['NumberOfDependents'])] = dependents_null_list
df['NumberOfDependents'] = df['NumberOfDependents'].astype(int)

In [ ]:
df.isnull().sum()

# 三、使用XGBoost构建分类器，使用交叉验证法对模型进行评估，使用网格搜索法进行超参数调节

In [ ]:
#XGBoot
y_train = df['SeriousDlqin2yrs'].astype('uint8')
df = df.drop(['SeriousDlqin2yrs'], axis=1)
x_train = df.values
#x_test = test_df.values

def Tuning(cv_params, other_params):
    model2 = xgb.XGBClassifier(**other_params)
    optimized_GBM = GridSearchCV(estimator=model2,
                                param_grid=cv_params,
                                cv=3,
                                n_jobs=4)
    optimized_GBM.fit(x_train, y_train)
    print(optimized_GBM.best_params_)
    print(optimized_GBM.best_score_)
    



In [ ]:
#目前最高0.7889133333333334
#GridSearchCV
cv_params = {'n_estimator':range(6,30,2)}
other_params = {
    'eta':0.3,
    'n_estimators':600,
    'max_depth':4,
    'min_child_weight':1,
    'gamma':0.2,
    'subsample':0.5,
    'colsample_bytree':1,
    'objective':'binary:logistic'
}
Tuning(cv_params, other_params)


In [ ]:
x_test = test_df.drop(['SeriousDlqin2yrs'],axis=1).values

In [ ]:
trainunnamed = df['Unnamed: 0']
Unnamed = test_df['Unnamed: 0']

gbm = xgb.XGBClassifier(
 n_estimators= 600,
 max_depth= 4,
 min_child_weight= 2,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1).fit(x_train, y_train)
xgb_predictions = gbm.predict_proba(x_test)[:,1]
StackingSubmission = pd.DataFrame({'Id':Unnamed, 'Probability':xgb_predictions})
StackingSubmission.to_csv("XGB.csv", index=False)
StackingSubmission.head()